In [ ]:
!pip install ultralytics kaggle tqdm albumentations

In [ ]:
!git clone https://github.com/n24q02m/VehicleDetection.git
%cd VehicleDetection

In [ ]:
import os
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
from kaggle_secrets import UserSecretsClient

In [ ]:
# Set up Kaggle credentials
os.makedirs('/root/.config/kaggle', exist_ok=True)

user_secrets = UserSecretsClient()
kaggle_username = user_secrets.get_secret("kaggle_username")
kaggle_key = user_secrets.get_secret("kaggle_key")

import json
token = {"username": kaggle_username, "key": kaggle_key}
with open('/root/.config/kaggle/kaggle.json', 'w') as f:
    json.dump(token, f)

!chmod 600 /root/.config/kaggle/kaggle.json

In [ ]:
# Download dataset from Kaggle
dataset_dir = "./data/soict-hackathon-2024_dataset"
dataset_name = "n24q02m/augmented-vehicle-detection-dataset"

os.makedirs(dataset_dir, exist_ok=True)
print(f"Downloading dataset {dataset_name}...")
kaggle.api.dataset_download_files(dataset_name, path=dataset_dir, unzip=True)
print("Dataset downloaded and extracted.")

In [ ]:
# Download pre-trained model from Kaggle
model_dir = "./runs/finetune_yolo11x/weights"
model_name = "n24q02m/finetuned-vehicle-detection-model"

os.makedirs(model_dir, exist_ok=True)
print(f"Downloading model {model_name}...")
kaggle.api.dataset_download_files(model_name, path=model_dir, unzip=True)
print("Pre-trained model downloaded.")

In [ ]:
!python train.py

In [ ]:
# Update final model on Kaggle
final_model_name = "n24q02m/final-vehicle-detection-model"
final_model_dir = "./runs/final-model"

metadata = {
    "title": "Final Vehicle Detection Model",
    "id": f"{final_model_name}",
    "licenses": [{"name": "CC0-1.0"}]
}

with open(os.path.join(final_model_dir, "dataset-metadata.json"), "w") as f:
    json.dump(metadata, f)
    
print(f"Updating model {final_model_name}...")
kaggle.api.dataset_create_version(final_model_dir, version_notes="Updated model")
print("Final model updated successfully")